# Runtime checking with pydantic

In [54]:
from typing import Literal, Optional, TypedDict, Union, List



### Dynamic Configuration


```
Consider a config file describing restaurants. 
Here is a list of configurable fields and constraints per restaurant:

• Name of the restaurant
    - The name must be less than 32 characters long, 
    - The name can only contain letters, numbers, quotation marks, and spaces.
• Owner's full name
• Address
• List of employees
    - There must be at least one chef and one server.
    - Each employee has a name and position.
    - Each employee either has a mailing address for a check or direct deposit details.
• List of dishes
    - Each dish has a name, price, and description. 
    - The name is limited to 16 characters, and the description is limited to 80 characters. 
    - Optionally, there is a picture (in the form of a filename) with each dish.
    - Each dish must have a unique name.
    - There must be at least three dishes on the menu.
• Number of seats
• Offers to-go orders (Boolean)
• Offers delivery (Boolean)
```


In [1]:
# From the last requirements, we can create a YAML file as follows

"""
name: Viafore's
owner: Pat Viafore
address: '123 Fake St. Fakington, FA 01234'
employees:
  - name: Pat Viafore
    position: Chef
    payment_details:
      bank_details:
        routing_number: '123456789'
    account_number: '123456789012'
  - name: Made-up McGee
    position: Server
    payment_details:
      bank_details:
        routing_number: '123456789'
        account_number: '123456789012'
  - name: Fabricated Frank
    position: Sous Chef
    payment_details:
      bank_details:
        routing_number: '123456789'
        account_number: '123456789012'
  - name: Illusory Ilsa
    position: Host
    payment_details:
      bank_details:
        routing_number: '123456789'
        account_number: '123456789012'
dishes:
  - name: Pasta and Sausage
    price_in_cents: 1295
    description: Rigatoni and sausage with a tomato-garlic-basil sauce
  - name: Pasta Bolognese
    price_in_cents: 1495
    description: Spaghetti with a rich tomato and beef Sauce
  - name: Caprese Salad
    price_in_cents: 795
    description: 'Tomato, buffalo mozzarella, and basil'
    picture: caprese.png
    number_of_seats: 12
to_go: true
delivery: false
"""

"\nname: Viafore's\nowner: Pat Viafore\naddress: '123 Fake St. Fakington, FA 01234'\nemployees:\n  - name: Pat Viafore\n    position: Chef\n    payment_details:\n      bank_details:\n        routing_number: '123456789'\n    account_number: '123456789012'\n  - name: Made-up McGee\n    position: Server\n    payment_details:\n      bank_details:\n        routing_number: '123456789'\n        account_number: '123456789012'\n  - name: Fabricated Frank\n    position: Sous Chef\n    payment_details:\n      bank_details:\n        routing_number: '123456789'\n        account_number: '123456789012'\n  - name: Illusory Ilsa\n    position: Host\n    payment_details:\n      bank_details:\n        routing_number: '123456789'\n        account_number: '123456789012'\ndishes:\n  - name: Pasta and Sausage\n    price_in_cents: 1295\n    description: Rigatoni and sausage with a tomato-garlic-basil sauce\n  - name: Pasta Bolognese\n    price_in_cents: 1495\n    description: Spaghetti with a rich tomato and 

Before continuing, install and import the 'yaml' library

In [5]:
"""
!pip install pyyaml
"""

'\n!pip install pyyaml\n'

In [6]:
import yaml

In [10]:
# Then we can use the 'yaml' library to read the last file and create a dictionary from it:

with open('restaurant.yml') as yaml_file:
    restaurant = yaml.safe_load(yaml_file)

restaurant


{'name': "Viafore's",
 'owner': 'Pat Viafore',
 'address': '123 Fake St. Fakington, FA 01234',
 'employees': [{'name': 'Pat Viafore',
   'position': 'Chef',
   'payment_details': {'bank_details': {'routing_number': '123456789'}},
   'account_number': '123456789012'},
  {'name': 'Made-up McGee',
   'position': 'Server',
   'payment_details': {'bank_details': {'routing_number': '123456789',
     'account_number': '123456789012'}}},
  {'name': 'Fabricated Frank',
   'position': 'Sous Chef',
   'payment_details': {'bank_details': {'routing_number': '123456789',
     'account_number': '123456789012'}}},
  {'name': 'Illusory Ilsa',
   'position': 'Host',
   'payment_details': {'bank_details': {'routing_number': '123456789',
     'account_number': '123456789012'}}}],
 'dishes': [{'name': 'Pasta and Sausage',
   'price_in_cents': 1295,
   'description': 'Rigatoni and sausage with a tomato-garlic-basil sauce'},
  {'name': 'Pasta Bolognese',
   'price_in_cents': 1495,
   'description': 'Spaghett

But how do we validate the constraints listed above?

In [12]:
# A first idea is to use a TypedDict, as follows

class AccountAndRoutingNumber(TypedDict):
    account_number: str
    routing_number: str

class BankDetails(TypedDict):
    bank_details: AccountAndRoutingNumber

AddressOrBankDetails = Union[str, BankDetails]
Position = Literal['Chef', 'Sous Chef', 'Host', 'Server', 'Delivery Driver']

class Dish(TypedDict):
    name: str
    price_in_cents: int
    description: str

class DishWithOptionalPicture(Dish, TypedDict, total=False):
    picture: str

class Employee(TypedDict):
    name: str
    position: Position
    payment_information: AddressOrBankDetails

class Restaurant(TypedDict):
    name: str
    owner: str
    address: str
    employees: list[Employee]
    dishes: list[Dish]
    number_of_seats: int
    to_go: bool
    delivery: bool


In [46]:
# With the last types, we can create the following function

def load_restaurant(filename: str) -> Restaurant:
    with open(filename) as yaml_file:
        return yaml.safe_load(yaml_file)


```
But there are issues with this approach:
• We can't control construction of a TypedDict, so we can't validate any fields as part of type construction.
• TypedDict cannot have additional methods on it.
•TypedDict doesn't do validation implicitly.
```

In [47]:
# For example, if we load a file with an invalid shape, the typechecker won't complain

restaurant = load_restaurant('Invalid_Restaurant.yml')
restaurant


{'invalid_name': 'This is the wrong file format'}

### Using pydantic

Before continuing, install and import the 'pydantic' library

In [20]:
"""
!pip install pydantic
"""

'\n!pip install pydantic\n'

In [67]:
from pydantic.dataclasses import dataclass
from pydantic import constr, conlist, validator, PositiveInt, StrictInt


In [49]:
# A better idea is to use pydantic to model our classes, as follows

@dataclass
class AccountAndRoutingNumber:
    account_number: str
    routing_number: str

@dataclass
class BankDetails:
    bank_details: AccountAndRoutingNumber

AddressOrBankDetails = Union[str, BankDetails]
Position = Literal['Chef', 'Sous Chef', 'Host', 'Server', 'Delivery Driver']

@dataclass
class Dish:
    name: str
    price_in_cents: int
    description: str
    picture: Optional[str] = None

@dataclass
class Employee:
    name: str
    position: Position
    payment_information: AddressOrBankDetails

@dataclass
class Restaurant:
    name: str
    owner: str
    address: str
    employees: list[Employee]
    dishes: list[Dish]
    number_of_seats: int
    to_go: bool
    delivery: bool


In [50]:
# To construct the pydantic type, we should change the load function as follows

def load_restaurant_2(filename: str) -> Restaurant:
    with open(filename) as yaml_file:
        data = yaml.safe_load(yaml_file)
        return Restaurant(**data)


In [51]:
# For example, if we load again the file with an invalid shape, pydantic will throw an exception

restaurant = load_restaurant_2('Invalid_Restaurant.yml')
restaurant


TypeError: __init__() got an unexpected keyword argument 'invalid_name'

### pydantic validators


In [ ]:
# We can use pydantic's constrained types, that check for specific constraints upon a field, as follows

@dataclass
class AccountAndRoutingNumber:
    account_number: constr(min_length=9, max_length=9)
    routing_number: constr(min_length=8, max_length=12)

@dataclass
class Address:
    address: constr(min_length=1)

@dataclass
class Dish:
    name: constr(min_length=1, max_length=16)
    price_in_cents: PositiveInt
    description: constr(min_length=1, max_length=80)
    picture: Optional[str] = None

@dataclass
class Restaurant:
    name: constr(regex=r'^[a-zA-Z0-9 ]*$', min_length=1, max_length=16)
    owner: constr(min_length=1)
    address: constr(min_length=1)
    employees: List[Employee]
    dishes: List[Dish]
    number_of_seats: PositiveInt
    to_go: bool
    delivery: bool


In [57]:
# We can also constrain lists to enforce further restrictions, as follows

@dataclass
class Restaurant:
    name: constr(regex=r'^[a-zA-Z0-9 ]*$', min_length=1, max_length=16)
    owner: constr(min_length=1)
    address: constr(min_length=1)
    employees: conlist(Employee, min_items=2)
    dishes: conlist(Dish, min_items=3)
    number_of_seats: PositiveInt
    to_go: bool
    delivery: bool


In [59]:
# We can also use custom validators to embed additional pieces of validation logic, as follows

@dataclass
class Restaurant:
    name: constr(regex=r'^[a-zA-Z0-9 ]*$', min_length=1, max_length=16)
    owner: constr(min_length=1)
    address: constr(min_length=1)
    employees: conlist(Employee, min_items=2)
    dishes: conlist(Dish, min_items=3)
    number_of_seats: PositiveInt
    to_go: bool
    delivery: bool
    
    @validator('employees')
    def check_chef_and_server(cls, employees):
        if (
            any(e for e in employees if e.position == 'Chef') and
            any(e for e in employees if e.position == 'Server')
        ):
            return employees
        
        raise ValueError('Must have at least one chef and one server')


### Validation Versus Parsing


In [61]:
# Pydantic is a parsing library, so it will coerce input data into the types you defined

@dataclass
class Model:
    value: int


In [65]:
# You can pass any data type into this model, as long as it can be converted to int

Model(value="123")

Model(value=123)

In [66]:
Model(value=5.5) # Truncates the value to 5


Model(value=5)

In [70]:
# But we can restrict this sort of behavior, as follows

@dataclass
class StrictModel:
    value: StrictInt


In [71]:
# Now any invalid data will raise an exception, even if it can be coerced as int

StrictModel(value="123")

ValidationError: 1 validation error for StrictModel
value
  value is not a valid integer (type=type_error.integer)

In [72]:
StrictModel(value=5.5)

ValidationError: 1 validation error for StrictModel
value
  value is not a valid integer (type=type_error.integer)